In [1]:
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import os
import time
import requests

In [2]:
driver=webdriver.Chrome(executable_path= r"C:\Users\Shivam Ghildiyal\drivers\chromedriver.exe")

### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [10]:
driver.get('https://www.amazon.in/')

In [13]:
search = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
user_demand = input('Enter the product to search')

Enter the product to search shoes


In [14]:
search.send_keys(user_demand)
driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]').click()

### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. 

Details to be scraped are: 
* "Brand Name",
* "Name of the Product",
* "Rating",
* "No. of Ratings", 
* "Price",
* "Return/Exchange",
* "Expected Delivery", 
* "Availability",
* "Other Details" 
* “Product URL”. 

In case, if any of the details are missing for any of the product then replace it by “-“.¶

In [15]:
links = []
for i in range(0,3):
    for l in driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]'):
        links.append(l.get_attribute('href'))

    try:
        driver.find_element_by_xpath('//li[@class="a-last"]').click()
    except:
        print('No pages left')
    time.sleep(5)

In [16]:
brand_name=[]
product_name=[]
rating=[]
n_rating=[]
price=[]
exreturn=[]
e_delivery=[]
available=[]
ot_details=[]

In [17]:
for l in links[0:15]:
    driver.get(l)
    
    try:
        brand_name.append(driver.find_element_by_xpath('//td[@class="a-span9"]/span[1]').text)
    except NoSuchElementException:
        brand_name.append(driver.find_element_by_xpath('//div[@id="bylineInfo_feature_div"]').text)
        
    try:
        product_name.append(driver.find_element_by_xpath('//span[@id="productTitle"]').text)
    except NoSuchElementException:
        product_name.append('none')
           
    try:
        rating.append(driver.find_element_by_xpath('//span[@id="acrPopover"]').get_attribute('title'))
        #rating.append((driver.find_element_by_xpath('//td[@class="a-size-base"]').text.split('\n')[1])             
    except NoSuchElementException:
        rating.append('none')    

    try:
        n_rating.append(driver.find_element_by_id("acrCustomerReviewText").text.split(' ')[0])
    except NoSuchElementException:
        n_rating.append('none') 
                      
    try:
        price.append(driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text)
    except NoSuchElementException:
        price.append('none')    

    try:
        exreturn.append(driver.find_element_by_xpath('//div[@id="availability"]/span').text)
    except NoSuchElementException:
        exreturn.append('none')
                      
    try:
        e_delivery.append(driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException:
        e_delivery.append('none')                      
                               
    try:
        available.append(driver.find_element_by_xpath('//div[@id="availability"]/span').text)
    except NoSuchElementException:
        available.append('none')  
    try:
        ot_details.append(driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]').text)
    except NoSuchElementException:
        ot_details.append('none')                      

        
    time.sleep(5)
driver.close()

In [18]:
brand_name

['Visit the Puma Store',
 'Visit the Bourge Store',
 'Visit the Bourge Store',
 'Visit the Bourge Store',
 'Brand: T-Rock',
 'Visit the Ethics Store',
 'Brand: Bersache',
 'Brand: AVIEMORE',
 'Brand: AVIEMORE',
 'Visit the Bourge Store',
 'Brand: Centrino',
 'Brand: AVIEMORE',
 'Fabric',
 'Visit the Bourge Store',
 'Brand: Sparx']

In [19]:
extra_words =['Visit', 'the', 'Store', 'Brand:']
for i in range(len(brand_name)):
    temp = brand_name[i].split(' ')
    for j in temp:
        if j not in extra_words:
            r = ''.join(j)
    brand_name[i] = r

In [20]:
seg = pd.DataFrame({})

seg['Brand']=brand_name
seg['Name']=product_name
seg['Rating']=rating
seg['Number of ratings']=n_rating
seg['price']=price
seg['exchange or return']=exreturn
seg['Expected Delivery']=e_delivery
seg['Availability']=available
seg['Other Details']=ot_details
seg['Product Link'] = links[0:15]

In [26]:
import numpy as np
seg.replace('none', np.nan, inplace = True)

In [28]:
seg

,Brand,Name,Rating,Number of ratings,price,exchange or return,Expected Delivery,Availability,Other Details,Product Link
0,Puma,Puma Men's Rapid Runner IDP Sneaker,4.2 out of 5 stars,433,"₹ 2,199.00",In stock.,"Tuesday, March 16",In stock.,Sole: Rubber\nClosure: Lace-Up\nShoe Width: Me...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Bourge,Bourge Men's Loire-z126 Running Shoes,4.0 out of 5 stars,"3,443",₹ 549.00,In stock.,"Tuesday, March 16",In stock.,Sole: Ethylene Vinyl Acetate\nClosure: Slip-On...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Bourge,Bourge Men's Vega-z1 Running Shoes,4.3 out of 5 stars,"5,439",₹ 625.00,In stock.,"Tuesday, March 16",In stock.,Closure: Lace-Up\nShoe Width: Medium\nMaterial...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Bourge,Bourge Men's Loire-z1 Running Shoes,4.2 out of 5 stars,"2,623",₹ 649.00,Available to ship in 1-2 days.,"Wednesday, March 17",Available to ship in 1-2 days.,Sole: Ethylene Vinyl Acetate\nClosure: Lace-Up...,https://www.amazon.in/gp/slredirect/picassoRed...
4,T-Rock,T-Rock Men's Running Shoes,2.9 out of 5 stars,"1,662",₹ 305.00 - ₹ 499.00,NaN,NaN,NaN,Sole: Polyvinyl Chloride\nClosure: Lace-Up\nSh...,https://www.amazon.in/T-Rock-Mens-Sports-Shoes...
5,Ethics,Ethics Men's Running Shoes,2.4 out of 5 stars,4,₹ 309.00 - ₹ 339.00,NaN,NaN,NaN,Sole: Polyvinyl Chloride\nClosure: Lace-Up\nSh...,https://www.amazon.in/Ethics-Mens-Running-Shoe...
6,Bersache,Bersache Men's Running Shoe (Set of 3 Pairs),2.9 out of 5 stars,20,₹ 587.00,NaN,NaN,NaN,Sole: Polyvinyl Chloride\nClosure: Lace-Up\nSh...,https://www.amazon.in/Bersache-Combo-Sports-Ru...
7,AVIEMORE,AVIEMORE Men's Greenwich-21 Running Shoes,5.0 out of 5 stars,4,₹ 599.00,In stock.,"Tuesday, March 16",In stock.,Closure: Slip-On\nShoe Width: Regular\nOuter M...,https://www.amazon.in/AVIEMORE-Greenwich-21-Ru...
8,AVIEMORE,AVIEMORE Men's Greenwich-23 Running Shoes,4.0 out of 5 stars,1,₹ 599.00,In stock.,"Tuesday, March 16",In stock.,Closure: Slip-On\nShoe Width: Regular\nOuter M...,https://www.amazon.in/AVIEMORE-Greenwich-23-Ru...
9,Bourge,Bourge Men's Vega-z2 Running Shoes,4.2 out of 5 stars,"5,511",₹ 625.00,In stock.,"Tuesday, March 16",In stock.,Closure: Slip-On\nShoe Width: Regular\nOuter M...,https://www.amazon.in/Bourge-Vega-4-Running-Sh...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [34]:
driver=webdriver.Chrome(executable_path= r"C:\Users\Shivam Ghildiyal\drivers\chromedriver.exe")
url = '''https://www.google.com/'''
driver.get(url)

In [35]:
driver.find_element_by_xpath('//*[@id="gb"]/div/div[1]/div/div[2]/a').click()
search = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
search_input= input('input to search')

input to search fruits


In [36]:
search.clear()
search.send_keys(search_input)
search.submit()

In [38]:
#We will now scroll down so more images can load 
driver.execute_script("window.scrollTo(0,25000);")

In [39]:
#tag of the of the all images
images=driver.find_elements_by_tag_name('img')

In [45]:
# now we will get the link of the images by get_attribute function 
images=[image.get_attribute('data-src') for image in images]

In [74]:
t =[]
for i in images:
    if i != None:
        t.append(i)
        
images = t.copy()

In [91]:
path = r"C:\Users\Shivam Ghildiyal\Desktop\ML\Topics\Web scrapping"
os.mkdir(path+'\\'+search_input)
for i,image in enumerate(images[0:100]):
    file = search_input+ str(i)+'.jpg'
    final =path + "\\" + search_input + '\\' + file
    with open(final,'wb')as f:       #writting a new file
        im=requests.get(image)                  #getting data from server of each link
        f.write(im.content)                      #then writing the byte content 

# for search word Car and Machine learning

In [108]:
driver=webdriver.Chrome(executable_path= r"C:\Users\Shivam Ghildiyal\drivers\chromedriver.exe")
url = '''https://www.google.com/'''
driver.get(url)

In [109]:
driver.find_element_by_xpath('//*[@id="gb"]/div/div[1]/div/div[2]/a').click()
search = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
search_input= input('input to search')

input to search machinelearning


In [110]:
search.clear()
search.send_keys(search_input)
search.submit()

In [111]:
#We will now scroll down so more images can load 
driver.execute_script("window.scrollTo(0,25000);")

In [112]:
#tag of the of the all images
images=driver.find_elements_by_tag_name('img')

In [113]:
# now we will get the link of the images by get_attribute function 
images=[image.get_attribute('data-src') for image in images]

In [114]:
t =[]
for i in images:
    if i != None:
        t.append(i)
        
images = t.copy()

In [115]:
path = r"C:\Users\Shivam Ghildiyal\Desktop\ML\Topics\Web scrapping"
os.mkdir(path+'\\'+search_input)
for i,image in enumerate(images[0:100]):
    file = search_input+ str(i)+'.jpg'
    final =path + "\\" + search_input + '\\' + file
    with open(final,'wb')as f:       #writting a new file
        im=requests.get(image)                  #getting data from server of each link
        f.write(im.content)                      #then writing the byte content 

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [3]:
driver=webdriver.Chrome(executable_path= r"C:\Users\Shivam Ghildiyal\drivers\chromedriver.exe")
url = '''https://www.flipkart.com/'''
driver.get(url)

In [4]:
#closing the login prompt
driver.find_element_by_xpath('/html/body/div[2]/div/div/button').click()

In [7]:
search = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
search.send_keys('smartphones')
driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/button').click()
search.submit()

In [8]:
links = []
for i in driver.find_elements_by_xpath('//a[@class="_1fQZEK"]'):
    links.append(i.get_attribute('href'))

In [64]:
brand= []
name = []
color = []
ram =[]
rom = []
p_cam = []
s_cam =[]
d_size = []
d_res = []
process = []
cores = []
battery = []
price =[]
for link in links:
    driver.get(link)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()    
    time.sleep(5)             
    try:
        brand.append(driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a').text.split(' ')[0])
    except NoSuchElementException:
        brand.append('-')
    
    try:
         name.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table[@class="_14cfVK"]//tr[3]//li[1]').text)
    except NoSuchElementException:
         name.append('-')

    try:
        color.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table[@class="_14cfVK"]//tr[4]//li[1]').text)
    except NoSuchElementException:
        color.append('-')
        
    try:
        ram.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[@class="_14cfVK"]//tr[2]//td[2]').text)
    except NoSuchElementException:
        ram.append('-')
    
    try:
         rom.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[@class="_14cfVK"]//tr[1]//td[2]').text)
    except NoSuchElementException:
         rom.append('-')

    try:
        p_cam.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[@class="_14cfVK"]//tr[2]//td[2]').text)
    except NoSuchElementException:
        p_cam.append('-')

    try:
        temp = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[@class="_14cfVK"]//tr[5]//td[2]').text
        if (temp != 'Yes'):
            s_cam.append(temp)
        else:
            s_cam.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[@class="_14cfVK"]//tr[6]//td[2]').text)
    except NoSuchElementException:
        s_cam.append('-')
        
    try:
        d_size.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[@class="_14cfVK"]//tr[1]//td[2]').text)
    except NoSuchElementException:
        d_size.append('-')
    
    try:
         d_res.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[@class="_14cfVK"]//tr[2]//td[2]').text)
    except NoSuchElementException:
         d_res.append('-')    
            
    try:
        cores.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[@class="_14cfVK"]//tr[3]//td[2]').text)
    except NoSuchElementException:
        cores.append('-')

    try:
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]//div[@class="_30jeq3 _16Jk6d"]').text)
    except NoSuchElementException:
        price.append('-')

    try:
        process.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[@class="_14cfVK"]//tr[2]//td[2]').text)
    except NoSuchElementException:
        process.append('-')
    
    try:
        temp = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table[@class="_14cfVK"]//tr[1]//td[2]').text
        if (temp != 'Yes'):
            battery.append(temp)
        else:
            battery.append(driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table[@class="_14cfVK"]//tr[1]//td[2]').text)
    except NoSuchElementException:
         battery.append('-')

In [65]:
q4 = pd.DataFrame(list(zip(brand,name,color,ram,rom,p_cam,s_cam,d_size,d_res,process,cores,battery,price,links)), 
    columns =['Brand', 'Product','Color', 'RAM Size', 'ROM Size', 'Primary Camera', 'Secondary Camera',
              'Screen size', 'Screen Resolution', 'Processor', 'Cores', 'Battery', 'Price','Product link'])

In [69]:
q4.to_csv('Mobile_phones.csv')
q4.head()

,Brand,Product,Color,RAM Size,ROM Size,Primary Camera,Secondary Camera,Screen size,Screen Resolution,Processor,Cores,Battery,Price,Product link
0,POCO,C3,Arctic Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh,"₹6,999",https://www.flipkart.com/poco-c3-arctic-blue-3...
1,POCO,M2,Pitch Black,6 GB,64 GB,13MP + 8MP + 5MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5000 mAh,"₹9,999",https://www.flipkart.com/poco-m2-pitch-black-6...
2,POCO,C3,Matte Black,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh,"₹6,999",https://www.flipkart.com/poco-c3-matte-black-3...
3,POCO,M3,Power Black,6 GB,128 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,999",https://www.flipkart.com/poco-m3-power-black-1...
4,POCO,X3,Cobalt Blue,6 GB,128 GB,64MP + 13MP + 2MP + 2MP,20MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 732G,Octa Core,6000 mAh,"₹15,499",https://www.flipkart.com/poco-x3-cobalt-blue-1...


### 5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [7]:
driver=webdriver.Chrome(executable_path= r"C:\Users\Shivam Ghildiyal\drivers\chromedriver.exe")
url='https://www.google.com/maps'
driver.get(url)

In [8]:
location = input('Enter the city to search')

Enter the city to search  Dehradun


In [9]:
search =driver.find_element_by_xpath('//*[@id="searchboxinput"]')
search.send_keys(location)
driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]').click()

In [11]:
#now we will get xpath of the map by clicking F12 
c=driver.find_element_by_xpath('//*[@id="scene"]/div[3]')

In [12]:
# as we need to right click on map so we use action chains
action=ActionChains(driver)

In [13]:
#now using context click we will perform the right click on the map
action.context_click(c).perform()

In [ ]:
#now after right click a list appear where we need to click on 'What is here?' option to get coordinates
driver.find_element_by_xpath("//li[@data-index='3']").click()

In [157]:
#after clicking we will see at bottom the coordinates so we will inspect and get the xpath of it and from that path we will extract text
coordinate=driver.find_element_by_xpath("//button[@vet='10547']").text

In [158]:
#finally we have coordinates
coordinate

'30.329687, 77.942027'

In [165]:
#spillting coodinates
coordinate=coordinate.split(',',1)

In [166]:
#assigning the coordinates of latitude and longitude 
latitude=coordinate[0]
longitude=coordinate[1]

In [168]:
#printing the result
print('latitude =',latitude)
print('longitude =',longitude)

latitude = 30.329687
longitude =  77.942027


## 6. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [75]:
url ='https://www.digit.in/'
driver.get(url)

In [76]:
driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a').click()

In [142]:
os =[]
display = []
process = []
memory = []
weight = []
dimen = []
g_process = []
laptop = []
name = []
price = []

for l in driver.find_elements_by_xpath('//table[@id="summtable"]//tr/td'):
    laptop.append(l.text)
i=0
while i < 28:
    name.append(laptop[i])
    price.append(laptop[i+2])
    i = i+3

for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[3]'):
    os.append(l.text.replace('OS :', ''))
    
for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[4]'):
    display.append(l.text.replace('Display :', ''))

for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[5]'):
    process.append(l.text.replace('Processor :', ''))

for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[6]'):
    memory.append(l.text.replace('Memory :', ''))
    
for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[7]'):
    weight.append(l.text.replace('Weight :', ''))    
    
for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[8]'):
    dimen.append(l.text.replace('Dimension :', ''))    

for l in driver.find_elements_by_xpath('//div[@class="Spcs-details"]//tr[9]'):
    g_process.append(l.text.replace('Graphics Processor :', '')) 


In [150]:
q7 = pd.DataFrame(list(zip(name, os,display,process,memory,weight,dimen,g_process,price)), 
               columns =["Name",'Operating System','Display', 'Processor','Memory', 'Weight', 'Dimension', 'Graphics','Price'])

In [152]:
q7

,Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics,Price
0,MSI GT76 TITAN DT 9SG,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,₹379990
1,Alienware 17 Area-51m,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,₹489909
2,HP Omen 15 2020,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,₹117790
3,Asus Zephyrus G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,₹164990
4,Lenovo Legion Y540,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,₹79990
5,Asus ROG Zephyrus G GA502,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,₹79990
6,Asus ROG Zephyrus S GX531,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),₹146000
7,MSI GT83VR 7RE Titan SLI,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3....,1.5 TB SATA/64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070,₹349990
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,₹259990
9,Dell G3,Windows 10 Home Plus,15.6 MP | NA,8th Gen Intel core i5-8300H | 2.3GHz,1TB HDD/8GB DDR4,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050,₹73900
